<a href="https://colab.research.google.com/github/moqri/deep_cell/blob/master/code_demo_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# load the libraries 
import numpy as np
import pandas as pd
import h5py
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.ndimage.filters import gaussian_filter
from sklearn.model_selection import train_test_split
from numpy import savetxt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import MaxPooling2D
from keras import backend as K
from keras.models import Model

import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import MaxPooling2D
from keras import backend as K
from keras.models import Model

from sklearn.model_selection import train_test_split
import pickle

import numpy as np

from keras.layers import InputLayer

from keras.layers import Conv2D

from keras.layers import Input, Dense


Using TensorFlow backend.


In [0]:
folder='dendritic'
expressions_url='https://mousescexpression.s3.amazonaws.com/dendritic_cell.h5'
genes_url='https://raw.githubusercontent.com/moqri/deep_cell/master/dendritic/genes.txt'
labels_url='https://raw.githubusercontent.com/moqri/deep_cell/master/dendritic_gene_pairs_200.txt'

In [3]:
#download the data
!mkdir {folder}
!wget -O {folder}/exprMatrix.h5 {expressions_url}
!wget -O {folder}/genes.txt {genes_url}
!wget -O {folder}/labels.txt {labels_url}

mkdir: cannot create directory ‘dendritic’: File exists
--2020-03-11 23:06:24--  https://mousescexpression.s3.amazonaws.com/dendritic_cell.h5
Resolving mousescexpression.s3.amazonaws.com (mousescexpression.s3.amazonaws.com)... 52.216.243.108
Connecting to mousescexpression.s3.amazonaws.com (mousescexpression.s3.amazonaws.com)|52.216.243.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 675899084 (645M) [application/x-www-form-urlencoded]
Saving to: ‘dendritic/exprMatrix.h5’

dendritic/exprMatri 100%[===================>] 644.59M  76.2MB/s    in 9.0s    

2020-03-11 23:06:34 (71.8 MB/s) - ‘dendritic/exprMatrix.h5’ saved [675899084/675899084]

--2020-03-11 23:06:34--  https://raw.githubusercontent.com/moqri/deep_cell/master/dendritic/genes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.


In [4]:
genes= pd.read_table(folder+"/genes.txt",index_col=1,header=None,names=['gene', 'id'],delimiter=' ')
genes.head()

,gene
id,
16334,ins2
19241,tmsb4x
16333,ins1
67945,rpl41
11461,actb


In [5]:
# clean the data :

#load the matrix
expression_df = pd.read_hdf(folder+"/exprMatrix.h5",index_col=0)

#clean the data

expression_df.index.rename('cell_id',inplace=1)
expression_df.shape

gene_names=[genes['gene'].ix[id] for id in expression_df.columns.values]
expression_df.columns=gene_names

expression_df.head()

expression_df=expression_df.loc[:, (expression_df != expression_df.iloc[0]).any()]  # remove constant columns
expression_df.shape

expression_df=expression_df[(expression_df.T != 0).any()] # remove rows of zeros
expression_df.shape


gene_count=expression_df.astype(bool).sum(axis=0)
cells=expression_df.count()
expression_df=expression_df[gene_count[gene_count>cells/10].index]
expression_df.shape

expression_df.head()

expression_df=(100*expression_df.transpose() / expression_df.sum(1)).round(2).transpose()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


In [6]:
#cell*genes
expression_df.shape

(4124, 13042)

In [0]:
# this function takes x and y and returns the image, number of bins determine the resolution. We will use this function later 
# in our construction for the image

def myplot(x, y, s, bins=150):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins)
    heatmap = gaussian_filter(heatmap, sigma=s)
    return heatmap.T 

In [0]:
# this function to read the file "pairs" and the the label assocaited with these pairs...example of what this function does 
# is in the next cell

def load_point_cloud_from_hd(path):
     entiregraph=[]
     y=[]
     with open(path) as f:
         for line in f:
             numbers_str = line.split()
             numbers_float = [str(x) for x in numbers_str]  #map(float,numbers_str) works too
             val=float(numbers_float[-1])
             if val!=2:          
                 y.append(val)
                 entiregraph.append(numbers_float[:-1])
             else:
                 y.append(1.0)
                 entiregraph.append(numbers_float[:-1])

                 
     return np.array(entiregraph),y


In [9]:
labels= pd.read_table(folder+"/labels.txt",index_col=False,header=None,names=['gene1', 'gene2','value'],delimiter='\t')
labels.head()

,gene1,gene2,value
0,stat3,tonsl,1
1,tonsl,stat3,2
2,stat3,exd1,0
3,stat3,tet2,1
4,tet2,stat3,2


In [0]:
pairs,y_=load_point_cloud_from_hd(folder+"/labels.txt")

In [11]:
# print the pair and the label assocatied with it
print(pairs[0])
print(y_[0])

print(pairs[1])
print(y_[1])

print(pairs[2])
print(y_[2])

['stat3' 'tonsl']
1.0
['tonsl' 'stat3']
1.0
['stat3' 'exd1']
0.0


In [12]:
# we can use the pairs with the expression_df as follows:
# lets take the first pair :

pair0=pairs[0]
gene_a=pair0[0]
gene_b=pair0[1]

print(gene_a)
print(gene_b)

stat3
tonsl


In [13]:
# we can input these genes to the table expression_df
expression_df[[gene_a,gene_b]]

# be careful here not all pairs in the file "pairs" are in expression_df so one has to check first if the pair exists in 
# expression_df

,stat3,tonsl
cell_id,,
GSE48968_GSM1189043_SRR935907,0.0,0.0
GSE48968_GSM1189045_SRR935909,0.0,0.0
GSE48968_GSM1189047_SRR935911,0.0,0.0
GSE48968_GSM1189048_SRR935912,0.0,0.0
GSE48968_GSM1189049_SRR935913,0.0,0.0
...,...,...
GSE78779_GSM2076658_SRR3196081,0.0,0.0
GSE78779_GSM2076659_SRR3196082,0.0,0.0
GSE78779_GSM2076660_SRR3196083,0.0,0.0


In [14]:
#we can try differnt threshold :

d=expression_df[[gene_a,gene_b]]
d=d[d[gene_a]+d[gene_b]>0.0] #we only drop when both are zero (replaced * with +)
print(d)

# in my code I skipped the empty ones - We are not droping these anymore

                                stat3  tonsl
cell_id                                     
GSE48968_GSM1189050_SRR935914    0.01    0.0
GSE48968_GSM1189053_SRR935917    0.01    0.0
GSE48968_GSM1189055_SRR935919    0.01    0.0
GSE48968_GSM1189056_SRR935920    0.01    0.0
GSE48968_GSM1189062_SRR935926    0.01    0.0
...                               ...    ...
GSE48968_GSM1190864_SRR940158    0.01    0.0
GSE78779_GSM2076618_SRR3196041   0.01    0.0
GSE78779_GSM2076625_SRR3196048   0.01    0.0
GSE78779_GSM2076627_SRR3196050   0.01    0.0
GSE78779_GSM2076650_SRR3196073   0.01    0.0

[923 rows x 2 columns]


In [0]:
x=np.log2(d+1)[gene_a].values
y=np.log2(d+1)[gene_b].values  

In [16]:
print(x[1:10])

[0.01435529 0.01435529 0.01435529 0.01435529 0.01435529 0.01435529
 0.01435529 0.01435529 0.01435529]


In [17]:
print(y[1:10])

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
# now compute the image
img = myplot(x, y, 16,bins=150)

In [19]:
img

array([[0.54703559, 0.55015224, 0.55634182, ..., 0.00837807, 0.00836347,
        0.00835624],
       [0.54490705, 0.54801156, 0.55417702, ..., 0.00834544, 0.0083309 ,
        0.0083237 ],
       [0.5406748 , 0.54375515, 0.54987265, ..., 0.00828056, 0.00826613,
        0.00825899],
       ...,
       [0.00782079, 0.00779033, 0.00772977, ..., 0.        , 0.        ,
        0.        ],
       [0.00786196, 0.00783134, 0.00777046, ..., 0.        , 0.        ,
        0.        ],
       [0.00788264, 0.00785194, 0.0077909 , ..., 0.        , 0.        ,
        0.        ]])

In [0]:
# we need to do the above for every pair in expression_df and hence the following function :

def generate_data(expression_df,pairs,y_values,s=16,threshold=0.05,size=None):

    expression_df = expression_df.loc[:,~expression_df.columns.duplicated()]
    X=[]
    y_out=[]
    for pair,k in zip(pairs,y_values):
        
        if size!=None:
            
            if len(X)>size:
                break
        
        gene_a=pair[0]  
        gene_b=pair[1]
        
        if gene_a in expression_df and gene_b in expression_df : # check if the genes are in the table before plugging them in
            d=expression_df[[gene_a,gene_b]]
            d=d[d[gene_a]+d[gene_b]>threshold] # here we use the threshold 
        
        
            x=np.log2(d+1)[gene_a].values
            y=np.log2(d+1)[gene_b].values  


            
            if len(x)!=0:
                if len(x.shape)!=1:
                    
                    continue
                else:
                    
                      
                    img = myplot(x, y, s,bins=150) # create the image
                    
                    X.append(img)
                    y_out.append(k)
                    
                    
            else:
                
                    img = myplot(x, y, s,bins=150)  # create the image
             

                    X.append(img)
                    y_out.append(k)
              

        
    
    return np.array(X),np.array(y_out)

In [0]:
# running may take a while : (I choose the size to be 1000 for speed)
X,y=generate_data(expression_df,pairs,y_,s=16,threshold=0.00,size=1000)

In [22]:
# so now you have an image X and its assoicated binary label y
print("image")
print(X[0])
print("LABEL")
print(y[0])

image
[[0.54703559 0.55015224 0.55634182 ... 0.00837807 0.00836347 0.00835624]
 [0.54490705 0.54801156 0.55417702 ... 0.00834544 0.0083309  0.0083237 ]
 [0.5406748  0.54375515 0.54987265 ... 0.00828056 0.00826613 0.00825899]
 ...
 [0.00782079 0.00779033 0.00772977 ... 0.         0.         0.        ]
 [0.00786196 0.00783134 0.00777046 ... 0.         0.         0.        ]
 [0.00788264 0.00785194 0.0077909  ... 0.         0.         0.        ]]
LABEL
1.0


In [0]:
# you can save them to HD using the following command :

import pickle

with open('X_Data_all.pkl','wb') as f:
    pickle.dump(X, f)

with open('y_Data_all.pkl','wb') as f:
    pickle.dump(y, f)

In [0]:
# now define the NN model :

def define_model_1(num_classes=3,shape_input=(32, 32, 1)):


    inputs = Input(shape=shape_input)

    x=Conv2D(32, kernel_size=(3, 3),
                     activation='relu')(inputs)
    x=Conv2D(64, (3, 3), activation='relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Dropout(0.25)(x)
    x=Flatten()(x)
    x=Dense(128, activation='relu')(x)
    x=Dropout(0.5)(x)
    predictions=Dense(num_classes, activation='softmax',name='final_output')(x)


    model = Model(inputs=inputs, outputs=predictions)
    print("model is defined")    
    return model

In [0]:
# define other functions needed for splitting the data, training the model,etc :

def prepare_data(X,y):        
    
    img_rows, img_cols = 150, 150
    
    # the data, split between train and test sets
    x_train, x_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)


    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 0.49434793151473155
    x_test /= 0.49434793151473155
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, 2)
    y_test = keras.utils.to_categorical(y_test, 2)
    
    return x_train, y_train, x_test, y_test


def train_model(model, x_train, y_train,x_test, y_test, batch_size=128,epochs=30):


    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    print("training the model")
    history=model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print("done training the model")
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    
    return history

In [26]:
# split the data : 

x_train, y_train, x_test, y_test = prepare_data(X,y)

x_train shape: (800, 150, 150, 1)
800 train samples
201 test samples


In [27]:
# Define the model : 

# input is the resolution of the image--in our case it is 150 by 150

model=define_model_1(num_classes=2,shape_input=(150, 150, 1))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
model is defined


In [28]:
# check the summary
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 148, 148, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 73, 73, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 341056)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4365

In [29]:
# train the model :
# choose higher epochs for your training --i choose a lower number because of time
train_model(model, x_train, y_train,x_test, y_test,batch_size=128,epochs=1)



training the model
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 800 samples, validate on 201 samples
Epoch 1/1





800/800 [==============================] - 79s 99ms/step - loss: 1.1325 - acc: 0.6350 - val_loss: 0.4179 - val_acc: 0.7612
done training the model
Test loss: 0.41788725769934965
Test accuracy: 0.7611940299248814
